# 2025 세법 개정본 반영

In [ ]:
pip install jpype1

In [ ]:
%pip install tabula-py

In [ ]:
pip install --upgrade langchain chromadb

In [ ]:
pip install requests-cache

In [10]:
import os
import re
from tqdm import tqdm
from tabula import read_pdf
from langchain.document_loaders import PyMuPDFLoader
from langchain.schema import Document
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.cache import SQLiteCache
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore
from dotenv import load_dotenv

load_dotenv()

pdf_file = "data/tax_etc/연말정산_주택자금·월세액_공제의이해.pdf"

# SQLite 캐시 설정
SQLiteCache(".cache_prompt_etc1.sqlite")

# 임베딩 모델 초기화
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# 모든 PDF 처리
docs = []

# Tabula로 PDF에서 표 데이터 읽기
try:
    tables = read_pdf(pdf_file, pages="9-12,15-24,27-28,31-39,43-70", stream=True, multiple_tables=True) # pages="19-44,47-71,75-161"
    table_texts = [table.to_string(index=False, header=True) for table in tables]
except Exception as e:
    print(f"Tabula Error with {pdf_file}: {e}")
    table_texts = []

# PyMuPDFLoader로 텍스트 데이터 로드 및 페이지 범위 필터링링
try:
    text_loader = PyMuPDFLoader(pdf_file)
    texts = text_loader.load()

    for i, text in enumerate(texts):
        text.metadata["page"] = i + 1      

    page_ranges = [(9, 12), (15, 24), (27, 28), (31, 39), (43, 70)]
    texts = [
        text for text in texts
        if any(start <= text.metadata.get("page", 0) <= end for start, end in page_ranges)
    ]
except Exception as e:
    print(f"PyMuPDFLoader Error with {pdf_file}: {e}")
    texts = []

# 표 텍스트를 Document 형식으로 변환
table_docs = [
    Document(page_content=text, metadata={"source": f"{pdf_file} - Table {i + 1}"})
    for i, text in enumerate(table_texts)
]

    # 텍스트를 Document 형식으로 변환
text_docs = [
    Document(page_content=text.page_content, metadata={"source": pdf_file})
    for text in texts
]

    # 텍스트 문서와 표 문서를 결합
docs.extend(text_docs + table_docs)

# 정규식 패턴화
update_docs = []

pattern1 = (
    r"연말정산 주택자금･월세액 공제의 이해\n\d+|"  
    r"\bNaN\b"
)
pattern2 =r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])"
pattern3 = r"\s+"

for doc in docs:
    if doc.page_content:
        edit_content = re.sub(pattern1, "", doc.page_content)
        edit_content = re.sub(pattern2, r"\1\2" , edit_content)
        edit_content = re.sub(pattern3, " ", edit_content)
    else:
        edit_content = ""
    
    updated_docs = Document(page_content=edit_content, metadata=doc.metadata)   
    update_docs.append(updated_docs)
    
# 텍스트를 청크로 분리
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o",
    chunk_size=2000,
    chunk_overlap=100,
)

split_docs = splitter.split_documents(update_docs)

# Vector Store 생성
vector_store = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding_model,
    collection_name="tax_etc",
    persist_directory="vector_store/chroma/tax_etc_4"
)

# Retriever 생성
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 10}
)

c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 1 of document data/tax_etc/연말정산_주택자금·월세액_공제의이해.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 3 of document data/tax_etc/연말정산_주택자금·월세액_공제의이해.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 5 of document data/tax_etc/연말정산_주택자금·월세액_공제의이해.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 7 of document data/tax_etc/연말정산_주택자금·월세액_공제의이해.pdf
  warnings.warn(
c:\Users\Playdat

In [8]:
docs[1]

Document(metadata={'source': 'data/tax_etc/연말정산_주택자금·월세액_공제의이해.pdf'}, page_content='연말정산 주택자금･월세액 공제의 이해\n4 ••\n참고\n주택자금공제\n종 류\n관련규정\n주택규모제한\n비 고\n주택임차차입금 원리금 상환액\n소법52④\n국민주택규모\n주거용 오피스텔 포함1)\n장기주택저당차입금 이자상환액\n소법52⑤\n제한없음2)\n주거용 오피스텔 제외\n1) 2013.8.13. 이후 최초로 원리금 상환액을 지급하는 분부터 적용하며, 국민주택규모에 해당하는 주거용 \n오피스텔을 말함\n2) 2024.1.1. 이후 차입하는 분부터 국민주택규모를 초과하는 주택도 공제 가능\n 세대와 세대주(소법 §52⑤ 1호, 소령 §112①)\n ⚫세대에는 거주자와 그 배우자(생계를 같이하는 경우 포함), 거주자와 같은 \n주소･거소에서 생계를 같이 하는 거주자와 그 배우자의 직계존비속(그 배우자들 \n포함) 및 형제자매를 모두 포함한다.\n ⚫세대주 여부의 판정은 과세기간 종료일 현재의 상황에 따른다.\n ⚫단독 세대주도 가능하며, 외국인 근로자의 경우에는 세대주 및 세대원이 될 수 없음\n 국민주택규모의 주택(소법 §52⑤ 1호, 소령 §112③)\n ⚫｢주택법｣에 따른 국민주택규모의 주택으로 세대당 주거전용면적이 85㎡ \n(수도권정비계획법 제2조 제1호에 따른 수도권을 제외한 도시지역이 아닌 읍 또는 \n면 지역은 100㎡)이하이고 그 부수되는 토지가 다음의 면적 이하인 상시 주거용 \n주택 및 그 부수토지를 말한다. 이 경우 해당 주택이 다가구주택이면 가구당 \n전용면적을 기준으로 한다.\n   - ｢국토의 계획 및 이용에 관한 법률｣ 제6조에 따른 도시지역의 토지:\n건물정착면적의 5배\n   - 그 밖의 토지: 건물정착면적의 10배\n2. 공제금액\n  원리금 상환금액의 40%를 근로소득금액에서 공제하되, 주택마련저축 공제금액과 \n주택임차차입금 원리금 상환액 공제금액의 합계액이 400만원을 초

In [9]:
update_docs[1]

Document(metadata={'source': 'data/tax_etc/연말정산_주택자금·월세액_공제의이해.pdf'}, page_content=' •• 참고주택자금공제종 류관련규정주택규모제한비 고주택임차차입금 원리금 상환액소법52④ 국민주택규모주거용 오피스텔 포함1) 장기주택저당차입금 이자상환액소법52⑤ 제한없음2) 주거용 오피스텔 제외 1) 2013.8.13. 이후 최초로 원리금 상환액을 지급하는 분부터 적용하며, 국민주택규모에 해당하는 주거용 오피스텔을 말함 2) 2024.1.1. 이후 차입하는 분부터 국민주택규모를 초과하는 주택도 공제 가능 세대와 세대주(소법 §52⑤ 1호, 소령 §112①) ⚫세대에는 거주자와 그 배우자(생계를 같이하는 경우 포함), 거주자와 같은 주소･거소에서 생계를 같이 하는 거주자와 그 배우자의 직계존비속(그 배우자들 포함) 및 형제자매를 모두 포함한다. ⚫세대주 여부의 판정은 과세기간 종료일 현재의 상황에 따른다. ⚫단독 세대주도 가능하며, 외국인 근로자의 경우에는 세대주 및 세대원이 될 수 없음 국민주택규모의 주택(소법 §52⑤ 1호, 소령 §112③) ⚫｢주택법｣에 따른 국민주택규모의 주택으로 세대당 주거전용면적이 85㎡ (수도권정비계획법 제2조 제1호에 따른 수도권을 제외한 도시지역이 아닌 읍 또는 면 지역은 100㎡)이하이고 그 부수되는 토지가 다음의 면적 이하인 상시 주거용 주택 및 그 부수토지를 말한다. 이 경우 해당 주택이 다가구주택이면 가구당 전용면적을 기준으로 한다. - ｢국토의 계획 및 이용에 관한 법률｣ 제6조에 따른 도시지역의 토지: 건물정착면적의 5배 - 그 밖의 토지: 건물정착면적의 10배 2. 공제금액 원리금 상환금액의 40%를 근로소득금액에서 공제하되, 주택마련저축 공제금액과 주택임차차입금 원리금 상환액 공제금액의 합계액이 400만원을 초과하는 경우 그 초과하는 금액은 없는 것으로 한다. 주택임차차입금 원리금 상환액 = MIN(㉠,㉡) ㉠ 주택마련저축 납입액 × 40% + 주택임차차입금 원리금 

In [11]:
# Prompt Template 생성
messages = [
        ("ai", """
        당신은 대한민국 세법에 대해 전문적으로 학습된 AI 도우미입니다. 저장된 세법 조항 데이터를 기반으로 사용자 질문에 답변하세요.

        - 모든 답변은 학습된 세법 데이터 내에서만 유효한 정보를 바탕으로 작성하세요. 데이터에 없는 내용은 추측하거나 임의로 생성하지 마세요.
        - 질문에 명확한 답변이 없거나 데이터 내에서 찾을 수 없는 경우, 정직하게 "잘 모르겠습니다."라고 말하고, 새로운 질문을 유도하세요.
        - 질문이 포함된 조항뿐 아니라, 필요 시 서로 연관된 다른 조항도 참고하여 답변의 정확성과 완성도를 높이세요.
        - 사용자가 이해하기 쉽게 답변을 구성하며, 중요한 키워드나 법 조항은 명확히 표시하세요.
        - 세법과 관련된 복잡한 질문에 대해서는 관련 조항 번호와 요약된 내용을 포함하여 답변을 제공하세요.
        
        추가 규칙:
        답변은 간결하고 명료하게 작성하되, 필요한 경우 관련 조항의 전문을 추가적으로 인용하세요.
        세법 용어를 사용자 친화적으로 설명하여 비전문가도 쉽게 이해할 수 있도록 하세요.
        질문을 완전히 이해하기 어렵거나 모호할 경우, 사용자가 구체적으로 질문을 다시 작성할 수 있도록 유도하는 후속 질문을 하세요.
        
        {context}")"""
        ),
        ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)
# 모델
model = ChatOpenAI(model="gpt-4o")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt 생성) -> model(정답) -> output parser
chain = {"context":retriever, "question": RunnablePassthrough()} | prompt_template | model | parser

In [ ]:
chain.invoke("총급여 68백만 원인 근로자의 신용카드 등 소득공제액은?")


'총급여 68백만 원인 근로자의 신용카드 등 소득공제는 "소득세법"에 따라 일정 기준을 충족해야 합니다. 일반적으로 신용카드 사용액의 일정 비율을 소득에서 공제 받을 수 있으며, 공제 한도는 총급여에 따라 다르게 설정됩니다.\n\n1. **기본 공제율**: 신용카드 사용금액의 15%를 공제 받을 수 있습니다. 다만, 전통시장 사용금액과 대중교통 사용금액은 각각 40%의 높은 공제율이 적용됩니다.\n\n2. **공제 한도**: 총급여 70백만 원 이하의 근로자는 최대 300만원까지 공제받을 수 있습니다.\n\n따라서, 총급여 68백만 원인 경우 최대 300만원 한도 내에서 신용카드 사용금액의 일정 비율을 소득에서 공제받을 수 있습니다. 정확한 공제금액은 실제 사용한 신용카드 금액과 그 중 전통시장 및 대중교통 사용금액에 따라 다르게 계산됩니다.\n\n정확한 공제금액을 계산하시려면 신용카드 사용 내역을 확인하시고, 각 항목에 대한 공제율을 적용하여 계산하셔야 합니다. 추가적인 정보 또는 세부 계산이 필요하시면 다시 문의해 주세요.'